# DX 704 Week 8 Project

This homework will modify a simulator controlling a small vehicle to implement tabular q-learning.
You will first test your code with random and greedy-epsilon policies, then tweak your own training method for a more optimal policy.

The full project description and a template notebook are available on GitHub: [Project 8 Materials](https://github.com/bu-cds-dx704/dx704-project-08).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Rover Simulator

The following Python class implements a simulation of a simple vehicle with integer x,y coordinates facing in one of 8 possible directions.


In [2]:
# DO NOT CHANGE

import random

class RoverSimulator(object):
    DIRECTIONS = ((0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1), (-1, 0), (-1, 1))

    def __init__(self, resolution):
        self.resolution = resolution
        self.terminal_state = self.construct_state(resolution // 2, resolution // 2, 0)

        self.initial_states = []
        for initial_x in (0, resolution // 2, resolution - 1):
            for initial_y in (0, resolution // 2, resolution - 1):
                for initial_direction in range(8):
                    initial_state = self.construct_state(initial_x, initial_y, initial_direction)
                    if initial_state != self.terminal_state:
                        self.initial_states.append(initial_state)

    def construct_state(self, x, y, direction):
        assert 0 <= x < self.resolution
        assert 0 <= y < self.resolution
        assert 0 <= direction < 8

        state = (y * self.resolution + x) * 8 + direction
        assert self.decode_state(state) == (x, y, direction)
        return state

    def decode_state(self, state):
        direction = state % 8
        x = (state // 8) % self.resolution
        y = state // (8 * self.resolution)

        return (x, y, direction)

    def get_actions(self, state):
        return [-1, 0, 1]

    def get_next_reward_state(self, curr_state, curr_action):
        if curr_state == self.terminal_state:
            # no rewards or changes from terminal state
            return (0, curr_state)

        (curr_x, curr_y, curr_direction) = self.decode_state(curr_state)
        (curr_dx, curr_dy) = self.DIRECTIONS[curr_direction]

        assert self.construct_state(curr_x, curr_y, curr_direction) == curr_state

        assert curr_action in (-1, 0, 1)

        next_x = min(max(0, curr_x + curr_dx), self.resolution - 1)
        next_y = min(max(0, curr_y + curr_dy), self.resolution - 1)
        next_direction = (curr_direction + curr_action) % 8

        next_state = self.construct_state(next_x, next_y, next_direction)
        next_reward = 1 if next_state == self.terminal_state else 0

        return (next_reward, next_state)

    def rollout_policy(self, policy_func, max_steps=1000):
        curr_state = self.sample_initial_state()
        for i in range(max_steps):
            curr_action = policy_func(curr_state, self.get_actions(curr_state))
            (next_reward, next_state) = self.get_next_reward_state(curr_state, curr_action)
            yield (curr_state, curr_action, next_reward, next_state)
            curr_state = next_state

    def sample_initial_state(self):
        return random.choice(self.initial_states)

In [3]:
simulator = RoverSimulator(16)
initial_sample = simulator.sample_initial_state()
print("INITIAL SAMPLE", initial_sample)

INITIAL SAMPLE 2044


## Part 1: Implement a Random Policy

Random policies are often used to test simulators and start initial exploration.
Implement a random policy for these simulators.

In [4]:
# YOUR CHANGES HERE

def random_policy(state, actions):
    return 0

Use the code below to test your random policy.
Then modify it to save the results in "log-random.tsv" with the columns curr_state, curr_action, next_reward and next_state.

In [5]:
# YOUR CHANGES HERE

for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=32):
    print("CURR STATE", curr_state, "ACTION", curr_action, "NEXT REWARD", next_reward, "NEXT STATE", next_state)

CURR STATE 1089 ACTION 0 NEXT REWARD 0 NEXT STATE 1225
CURR STATE 1225 ACTION 0 NEXT REWARD 0 NEXT STATE 1361
CURR STATE 1361 ACTION 0 NEXT REWARD 0 NEXT STATE 1497
CURR STATE 1497 ACTION 0 NEXT REWARD 0 NEXT STATE 1633
CURR STATE 1633 ACTION 0 NEXT REWARD 0 NEXT STATE 1769
CURR STATE 1769 ACTION 0 NEXT REWARD 0 NEXT STATE 1905
CURR STATE 1905 ACTION 0 NEXT REWARD 0 NEXT STATE 2041
CURR STATE 2041 ACTION 0 NEXT REWARD 0 NEXT STATE 2041
CURR STATE 2041 ACTION 0 NEXT REWARD 0 NEXT STATE 2041
CURR STATE 2041 ACTION 0 NEXT REWARD 0 NEXT STATE 2041
CURR STATE 2041 ACTION 0 NEXT REWARD 0 NEXT STATE 2041
CURR STATE 2041 ACTION 0 NEXT REWARD 0 NEXT STATE 2041
CURR STATE 2041 ACTION 0 NEXT REWARD 0 NEXT STATE 2041
CURR STATE 2041 ACTION 0 NEXT REWARD 0 NEXT STATE 2041
CURR STATE 2041 ACTION 0 NEXT REWARD 0 NEXT STATE 2041
CURR STATE 2041 ACTION 0 NEXT REWARD 0 NEXT STATE 2041
CURR STATE 2041 ACTION 0 NEXT REWARD 0 NEXT STATE 2041
CURR STATE 2041 ACTION 0 NEXT REWARD 0 NEXT STATE 2041
CURR STATE

Submit "log-random.tsv" in Gradescope.

In [6]:
import random
import pandas as pd


def random_policy(state, actions):
    return random.choice(actions)   


records = []

for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=32):
    records.append({
        "curr_state": curr_state,
        "curr_action": curr_action,
        "next_reward": next_reward,
        "next_state": next_state
    })


df = pd.DataFrame(records)
df.to_csv("log-random.tsv", sep="\t", index=False)


print(df.head())


   curr_state  curr_action  next_reward  next_state
0         126           -1            0         117
1         117           -1            0         108
2         108            0            0         108
3         108            0            0         108
4         108            0            0         108


## Part 2: Implement Q-Learning with Random Policy

The code below runs 32 random rollouts of 1024 steps using your random policy.
Modify the rollout code to implement Q-Learning.
Just implement one learning update for each sampled state-action in the simulation.
Use $\alpha=1$ and $\gamma=0.9$ since the simulator is deterministic and there is a sink where the rewards stop.




In [7]:
# YOUR CHANGES HERE

for episode in range(32):
    for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=1024):
        #print("CURR STATE", curr_state, "ACTION", curr_action, "NEXT REWARD", next_reward, "NEXT STATE", next_state)
        pass

Save each step in the simulator in a file "q-random.tsv" with columns curr_state, curr_action, next_reward, next_state, old_value, new_value.

In [8]:
# YOUR CHANGES HERE

import pandas as pd
from collections import defaultdict


alpha = 1.0    
gamma = 0.9     
Q = defaultdict(float)  

records = []


for episode in range(32):
    for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=1024):
        old_value = Q[(curr_state, curr_action)]

        next_actions = simulator.get_actions(next_state)
        next_q_values = [Q[(next_state, a)] for a in next_actions]
        best_next = max(next_q_values) if next_q_values else 0.0

        new_value = old_value + alpha * (next_reward + gamma * best_next - old_value)
        Q[(curr_state, curr_action)] = new_value

        records.append({
            "curr_state": curr_state,
            "curr_action": curr_action,
            "next_reward": next_reward,
            "next_state": next_state,
            "old_value": old_value,
            "new_value": new_value
        })


df = pd.DataFrame(records)
df.to_csv("q-random.tsv", sep="\t", index=False)

print("Q-learning log saved to q-random.tsv")




Q-learning log saved to q-random.tsv


Submit "q-random.tsv" in Gradescope.

## Part 3: Implement Epsilon-Greedy Policy

Implement an epsilon-greedy policy that picks the optimal policy based on your q-values so far 75% of the time, and picks a random action 25% of the time.
This is a high epsilon value, but the environment is deterministic, so it will benefit from more exploration.

In [9]:
# YOUR CHANGES HERE

# hard-code epsilon=0.25. this is high but the environment is deterministic.
def epsilon_greedy_policy(state, actions):
    return 0

Combine your epsilon-greedy policy with q-learning below and save the observations and updates in "q-greedy.tsv" with columns curr_state, curr_action, next_reward, next_state, old_value, new_value.

Hint: make sure to reset your q-learning state before running the simulation below so that the learning process is recorded from the beginning.

In [10]:
# YOUR CHANGES HERE

for episode in range(32):
    for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(epsilon_greedy_policy, max_steps=1024):
        #print("CURR STATE", curr_state, "ACTION", curr_action, "NEXT REWARD", next_reward, "NEXT STATE", next_state)

        if next_reward > 0:
            # moving to terminal state
            break

        pass

In [11]:
import numpy as np
import pandas as pd

Q = {}
gamma = 0.9
epsilon = 0.25  

def epsilon_greedy_policy(state, actions):
    if np.random.rand() < epsilon:             
        return int(np.random.choice(actions))
    q = np.array([Q.get((state, a), 0.0) for a in actions], float)
    best = np.flatnonzero(q == q.max())        
    return int(actions[np.random.choice(best)])

records = []
for episode in range(32):
    for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(
        epsilon_greedy_policy, max_steps=1024
    ):
        old = Q.get((curr_state, curr_action), 0.0)
        next_actions = simulator.get_actions(next_state)
        next_q = np.array([Q.get((next_state, a), 0.0) for a in next_actions], float)
        best_next = float(next_q.max()) if next_q.size else 0.0

        new = next_reward + gamma * best_next
        Q[(curr_state, curr_action)] = new

        records.append({
            "curr_state":  curr_state,
            "curr_action": int(curr_action),
            "next_reward": int(next_reward),
            "next_state":  next_state,
            "old_value":   old,
            "new_value":   new
        })

        if next_reward > 0:   
            break

pd.DataFrame(records).to_csv("q-greedy.tsv", sep="\t", index=False)


Submit "q-greedy.tsv" in Gradescope.

## Part 4: Extract Policy from Q-Values

Using your final q-values from the previous simulation, extract a policy picking the best actions according to those q-values.
Save the policy in a file "policy-greedy.tsv" with columns state and action.

In [12]:
# YOUR CHANGES HERE

...

Ellipsis

In [13]:
import numpy as np
import pandas as pd

qdata = pd.read_csv("q-greedy.tsv", sep="\t")

best_actions = (
    qdata.groupby("curr_state")
         .apply(lambda g: g.loc[g["new_value"].idxmax(), "curr_action"])
         .reset_index(name="action")
         .rename(columns={"curr_state": "state"})
)


best_actions.to_csv("policy-greedy.tsv", sep="\t", index=False)
print("Saved policy-greedy.tsv")
print(best_actions.head())


Saved policy-greedy.tsv
   state  action
0      3       0
1      4      -1
2      5      -1
3      6       0
4      7       0


/tmp/ipykernel_2421/1908227982.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.loc[g["new_value"].idxmax(), "curr_action"])


Submit "policy-greedy.tsv" in Gradescope.

## Part 5: Implement Large Policy

Train a more optimal policy using q-learning.
Save the policy in a file "policy-optimal.tsv" with columns state and action.

Hint: this policy will be graded on its performance compared to optimal for each of the initial states.
**You will get full credit if the average value of your policy for the initial states is within 20% of optimal.**
Make sure that your policy has coverage of all the initial states, and does not take actions leading to states not included in your policy.
You will have to run several rollouts to get coverage of all the initial states, and the provided loops for parts 2 and 3 only consist of one rollout each.

Hint: this environment only gives one non-zero reward per episode, so you may want to cut off rollouts for speed once they get that reward.
But make sure you update the q-values first!

In [14]:
# YOUR CHANGES HERE

...

Ellipsis

In [15]:
import numpy as np
import pandas as pd


Q = {}                                  
gamma = 0.9
alpha = 1.0                              
eps_hi, eps_lo = 0.30, 0.05              
max_steps = 2048

def eps_greedy(state, actions, eps):
    if np.random.rand() < eps:                      
        return int(np.random.choice(actions))
    qs = np.array([Q.get((state, a), 0.0) for a in actions], float)
    best = np.flatnonzero(qs == qs.max())          
    return int(actions[np.random.choice(best)])

visited_states = set()
episodes = 60                    
per_pass  = len(simulator.initial_states)

for ep in range(episodes):
    eps = eps_hi + (eps_lo - eps_hi) * (ep / max(1, episodes - 1))

    for s0 in simulator.initial_states:
        curr_state = s0
        for t in range(max_steps):
            visited_states.add(curr_state)
            actions = simulator.get_actions(curr_state)
            a = eps_greedy(curr_state, actions, eps)

            r, next_state = simulator.get_next_reward_state(curr_state, a)
            visited_states.add(next_state)

            old = Q.get((curr_state, a), 0.0)
            next_actions = simulator.get_actions(next_state)
            best_next = 0.0 if not next_actions else max(Q.get((next_state, aa), 0.0) for aa in next_actions)
            Q[(curr_state, a)] = r + gamma * best_next

            curr_state = next_state
            if r > 0:       
                break
            if curr_state == simulator.terminal_state:
                break

def best_action_for_state(s):
    acts = simulator.get_actions(s)
    qs = np.array([Q.get((s, a), 0.0) for a in acts], float)
    return int(acts[int(np.flatnonzero(qs == qs.max())[0])])

all_states = sorted(visited_states | {simulator.terminal_state} | set(simulator.initial_states))
policy_rows = [{"state": s, "action": (0 if s == simulator.terminal_state else best_action_for_state(s))}
               for s in all_states]

pd.DataFrame(policy_rows).to_csv("policy-optimal.tsv", sep="\t", index=False)
print("Saved policy-optimal.tsv with", len(policy_rows), "states")


Saved policy-optimal.tsv with 1996 states


Submit "policy-optimal.tsv" in Gradescope.

## Part 6: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 7: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

In [16]:
from pathlib import Path

ACK_FILE = Path("acknowledgments.txt")
ACK_TEXT = "My dad helped me with this"

def main():
    ACK_FILE.write_text(ACK_TEXT)
    print(f"Saved acknowledgments to: {ACK_FILE.resolve()}")

if __name__ == "__main__":
    main()

Saved acknowledgments to: /workspaces/dx704-project-08/acknowledgments.txt
